# Microsoft Agent Framework ব্যবহার করে Human-in-the-Loop Workflow

## 🎯 শেখার লক্ষ্য

এই নোটবুকে, আপনি Microsoft Agent Framework এর `RequestInfoExecutor` ব্যবহার করে **human-in-the-loop** workflow বাস্তবায়ন করতে শিখবেন। এই শক্তিশালী প্যাটার্নটি AI workflow-কে থামিয়ে মানব ইনপুট সংগ্রহ করার সুযোগ দেয়, যা এজেন্টকে ইন্টারঅ্যাকটিভ করে তোলে এবং গুরুত্বপূর্ণ সিদ্ধান্তে মানুষের নিয়ন্ত্রণ প্রদান করে।

## 🔄 Human-in-the-Loop কী?

**Human-in-the-loop (HITL)** একটি ডিজাইন প্যাটার্ন যেখানে AI এজেন্ট কার্যক্রম থামিয়ে মানব ইনপুটের অনুরোধ করে এবং তারপর কার্যক্রম চালিয়ে যায়। এটি গুরুত্বপূর্ণ কারণ:

- ✅ **গুরুত্বপূর্ণ সিদ্ধান্ত** - গুরুত্বপূর্ণ পদক্ষেপ নেওয়ার আগে মানব অনুমোদন নিন
- ✅ **অস্পষ্ট পরিস্থিতি** - AI যখন অনিশ্চিত থাকে তখন মানুষের কাছে স্পষ্টতা চাইতে দিন
- ✅ **ব্যবহারকারীর পছন্দ** - ব্যবহারকারীদের একাধিক অপশনের মধ্যে থেকে বেছে নিতে দিন
- ✅ **নিয়মাবলী ও নিরাপত্তা** - নিয়মিত কার্যক্রমে মানব তত্ত্বাবধান নিশ্চিত করুন
- ✅ **ইন্টারঅ্যাকটিভ অভিজ্ঞতা** - এমন কথোপকথনমূলক এজেন্ট তৈরি করুন যা ব্যবহারকারীর ইনপুটের ভিত্তিতে সাড়া দেয়

## 🏗️ Microsoft Agent Framework-এ এটি কীভাবে কাজ করে

ফ্রেমওয়ার্কটি HITL-এর জন্য তিনটি গুরুত্বপূর্ণ উপাদান প্রদান করে:

1. **`RequestInfoExecutor`** - একটি বিশেষ executor যা workflow থামিয়ে `RequestInfoEvent` প্রকাশ করে
2. **`RequestInfoMessage`** - মানুষের কাছে পাঠানো টাইপড অনুরোধ payload-এর জন্য বেস ক্লাস
3. **`RequestResponse`** - `request_id` ব্যবহার করে মানব প্রতিক্রিয়াকে মূল অনুরোধের সাথে সংযুক্ত করে

**Workflow Pattern:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 আমাদের উদাহরণ: ব্যবহারকারীর নিশ্চিতকরণের মাধ্যমে হোটেল বুকিং

আমরা শর্তাধীন workflow-এর উপর ভিত্তি করে মানব নিশ্চিতকরণ **প্রস্তাব করার আগে** একটি উদাহরণ তৈরি করব:

1. ব্যবহারকারী একটি গন্তব্যের অনুরোধ করে (যেমন, "প্যারিস")
2. `availability_agent` চেক করে রুম উপলব্ধ আছে কিনা
3. **যদি রুম না থাকে** → `confirmation_agent` জিজ্ঞাসা করে "আপনি কি বিকল্প দেখতে চান?"
4. Workflow **থামে** `RequestInfoExecutor` ব্যবহার করে
5. **মানব প্রতিক্রিয়া দেয়** "হ্যাঁ" বা "না" কনসোল ইনপুটের মাধ্যমে
6. `decision_manager` প্রতিক্রিয়ার ভিত্তিতে রুট নির্ধারণ করে:
   - **হ্যাঁ** → বিকল্প গন্তব্য দেখান
   - **না** → বুকিং অনুরোধ বাতিল করুন
7. চূড়ান্ত ফলাফল প্রদর্শন করুন

এটি দেখায় কীভাবে এজেন্টের প্রস্তাবের উপর ব্যবহারকারীদের নিয়ন্ত্রণ দেওয়া যায়!

---

চলুন শুরু করি! 🚀


## ধাপ ১: প্রয়োজনীয় লাইব্রেরি আমদানি করুন

আমরা স্ট্যান্ডার্ড Agent Framework এর উপাদানগুলো আমদানি করি, সাথে **মানব-ইন-দ্য-লুপ নির্দিষ্ট ক্লাসগুলো**:
- `RequestInfoExecutor` - এক্সিকিউটর যা মানব ইনপুটের জন্য ওয়ার্কফ্লো থামায়
- `RequestInfoEvent` - ইভেন্ট যা মানব ইনপুটের অনুরোধের সময় নির্গত হয়
- `RequestInfoMessage` - টাইপ করা অনুরোধ পে-লোডের জন্য বেস ক্লাস
- `RequestResponse` - অনুরোধের সাথে মানব প্রতিক্রিয়াগুলো সম্পর্কিত করে
- `WorkflowOutputEvent` - ওয়ার্কফ্লো আউটপুট সনাক্ত করার জন্য ইভেন্ট


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## ধাপ ২: কাঠামোগত আউটপুটের জন্য Pydantic মডেল সংজ্ঞায়িত করুন

এই মডেলগুলো এজেন্টরা যে **স্কিমা** ফেরত দেবে তা সংজ্ঞায়িত করে। আমরা শর্তাধীন ওয়ার্কফ্লো থেকে সমস্ত মডেল রাখি এবং যোগ করি:

**মানব-ইন-দ্য-লুপের জন্য নতুন:**
- `HumanFeedbackRequest` - `RequestInfoMessage` এর সাবক্লাস যা মানুষের কাছে পাঠানো অনুরোধ পে-লোড সংজ্ঞায়িত করে
  - এতে রয়েছে `prompt` (জিজ্ঞাসিত প্রশ্ন) এবং `destination` (অপ্রাপ্য শহরের প্রসঙ্গ)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## ধাপ ৩: হোটেল বুকিং টুল তৈরি করুন

শর্তাধীন ওয়ার্কফ্লো থেকে একই টুল - গন্তব্যস্থলে রুম উপলব্ধ কিনা তা পরীক্ষা করে।


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ধাপ ৪: রাউটিংয়ের জন্য শর্ত ফাংশন সংজ্ঞায়িত করা

আমাদের **চারটি শর্ত ফাংশন** প্রয়োজন আমাদের human-in-the-loop ওয়ার্কফ্লোর জন্য:

**শর্তযুক্ত ওয়ার্কফ্লো থেকে:**
1. `has_availability_condition` - রাউট করে যখন হোটেলগুলো উপলব্ধ থাকে
2. `no_availability_condition` - রাউট করে যখন হোটেলগুলো উপলব্ধ থাকে না

**human-in-the-loop এর জন্য নতুন:**
3. `user_wants_alternatives_condition` - রাউট করে যখন ব্যবহারকারী বিকল্পের জন্য "হ্যাঁ" বলে
4. `user_declines_alternatives_condition` - রাউট করে যখন ব্যবহারকারী বিকল্পের জন্য "না" বলে


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## ধাপ ৫: সিদ্ধান্ত ব্যবস্থাপক এক্সিকিউটর তৈরি করুন

এটি **মানব-ইন-দ্য-লুপ প্যাটার্নের মূল অংশ**! `DecisionManager` একটি কাস্টম `Executor`, যা:

1. **মানব প্রতিক্রিয়া গ্রহণ করে** `RequestResponse` অবজেক্টের মাধ্যমে
2. **ব্যবহারকারীর সিদ্ধান্ত প্রক্রিয়া করে** (হ্যাঁ/না)
3. **ওয়ার্কফ্লো পরিচালনা করে** সঠিক এজেন্টদের কাছে বার্তা পাঠিয়ে

মূল বৈশিষ্ট্য:
- `@handler` ডেকোরেটর ব্যবহার করে পদ্ধতিগুলোকে ওয়ার্কফ্লো ধাপ হিসেবে প্রকাশ করে
- `RequestResponse[HumanFeedbackRequest, str]` গ্রহণ করে, যেখানে মূল অনুরোধ এবং ব্যবহারকারীর উত্তর উভয়ই থাকে
- সহজ "হ্যাঁ" বা "না" বার্তা প্রদান করে, যা আমাদের শর্ত ফাংশনগুলোকে সক্রিয় করে


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## ধাপ ৬: কাস্টম ডিসপ্লে এক্সিকিউটর তৈরি করুন

শর্তাধীন ওয়ার্কফ্লো থেকে একই ডিসপ্লে এক্সিকিউটর - এটি চূড়ান্ত ফলাফলকে ওয়ার্কফ্লো আউটপুট হিসেবে প্রদান করে।


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## ধাপ ৭: পরিবেশ ভেরিয়েবল লোড করুন

LLM ক্লায়েন্ট (GitHub Models, Azure OpenAI, বা OpenAI) কনফিগার করুন।


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## ধাপ ৮: এআই এজেন্ট এবং এক্সিকিউটর তৈরি করুন

আমরা **ছয়টি ওয়ার্কফ্লো কম্পোনেন্ট** তৈরি করি:

**এজেন্ট (AgentExecutor-এ মোড়ানো):**
1. **availability_agent** - টুল ব্যবহার করে হোটেলের প্রাপ্যতা পরীক্ষা করে
2. **confirmation_agent** - 🆕 মানব নিশ্চিতকরণের অনুরোধ প্রস্তুত করে
3. **alternative_agent** - বিকল্প শহর প্রস্তাব করে (যখন ব্যবহারকারী হ্যাঁ বলে)
4. **booking_agent** - বুকিং উৎসাহিত করে (যখন রুম উপলব্ধ থাকে)
5. **cancellation_agent** - 🆕 বাতিলকরণের বার্তা পরিচালনা করে (যখন ব্যবহারকারী না বলে)

**বিশেষ এক্সিকিউটর:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` যা মানব ইনপুটের জন্য ওয়ার্কফ্লো থামায়
7. **decision_manager** - 🆕 কাস্টম এক্সিকিউটর যা মানব প্রতিক্রিয়ার ভিত্তিতে রাউট করে (উপরে ইতিমধ্যে সংজ্ঞায়িত)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## ধাপ ৯: মানব-ইন-দ্য-লুপ সহ ওয়ার্কফ্লো তৈরি করুন

এখন আমরা **শর্তসাপেক্ষ রাউটিং** সহ মানব-ইন-দ্য-লুপ পথ অন্তর্ভুক্ত করে ওয়ার্কফ্লো গ্রাফ তৈরি করি:

**ওয়ার্কফ্লো কাঠামো:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**মূল সংযোগসমূহ:**
- `availability_agent → confirmation_agent` (যখন কোনো রুম নেই)
- `confirmation_agent → prepare_human_request` (ধরন পরিবর্তন)
- `prepare_human_request → request_info_executor` (মানবের জন্য বিরতি)
- `request_info_executor → decision_manager` (সবসময় - RequestResponse প্রদান করে)
- `decision_manager → alternative_agent` (যখন ব্যবহারকারী "হ্যাঁ" বলে)
- `decision_manager → cancellation_agent` (যখন ব্যবহারকারী "না" বলে)
- `availability_agent → booking_agent` (যখন রুম উপলব্ধ থাকে)
- সব পথ `display_result` এ শেষ হয়


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## ধাপ ১০: টেস্ট কেস ১ চালান - শহর যেখানে উপলব্ধতা নেই (প্যারিস, মানব নিশ্চিতকরণ সহ)

এই টেস্টটি **সম্পূর্ণ মানব-ইন-দ্য-লুপ চক্র** প্রদর্শন করে:

1. প্যারিসে হোটেলের অনুরোধ করুন
2. availability_agent পরীক্ষা করে → কোনো রুম নেই
3. confirmation_agent মানবমুখী প্রশ্ন তৈরি করে
4. request_info_executor **ওয়ার্কফ্লো থামায়** এবং `RequestInfoEvent` প্রকাশ করে
5. **অ্যাপ্লিকেশন ইভেন্ট সনাক্ত করে এবং কনসোলে ব্যবহারকারীকে প্রম্পট দেয়**
6. ব্যবহারকারী "yes" বা "no" টাইপ করে
7. অ্যাপ্লিকেশন `send_responses_streaming()` এর মাধ্যমে উত্তর পাঠায়
8. decision_manager উত্তর অনুযায়ী রাউট করে
9. চূড়ান্ত ফলাফল প্রদর্শিত হয়

**মূল প্যাটার্ন:**
- প্রথম পুনরাবৃত্তির জন্য `workflow.run_stream()` ব্যবহার করুন
- পরবর্তী পুনরাবৃত্তির জন্য `workflow.send_responses_streaming(pending_responses)` ব্যবহার করুন
- মানব ইনপুট প্রয়োজন হলে সনাক্ত করতে `RequestInfoEvent` শুনুন
- চূড়ান্ত ফলাফল ধরতে `WorkflowOutputEvent` শুনুন


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## ধাপ ১১: টেস্ট কেস ২ চালান - শহর যেখানে রুম পাওয়া যাচ্ছে (স্টকহোম - কোনো মানবিক ইনপুট প্রয়োজন নেই)

এই টেস্টটি রুম পাওয়া গেলে **সরাসরি পথ** প্রদর্শন করে:

1. স্টকহোমে হোটেল অনুরোধ করুন
2. availability_agent পরীক্ষা করে → রুম পাওয়া যাচ্ছে ✅
3. booking_agent বুকিংয়ের পরামর্শ দেয়
4. display_result নিশ্চিতকরণ দেখায়
5. **কোনো মানবিক ইনপুট প্রয়োজন নেই!**

যখন রুম পাওয়া যায়, তখন পুরো প্রক্রিয়া মানবিক-ইনপুট পথকে সম্পূর্ণভাবে এড়িয়ে যায়।


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## মূল বিষয় এবং Human-in-the-Loop সেরা অনুশীলন

### ✅ আপনি যা শিখেছেন:

#### 1. **RequestInfoExecutor প্যাটার্ন**
Microsoft Agent Framework-এ Human-in-the-loop প্যাটার্ন তিনটি মূল উপাদান ব্যবহার করে:
- `RequestInfoExecutor` - ওয়ার্কফ্লো থামায় এবং ইভেন্ট প্রকাশ করে
- `RequestInfoMessage` - টাইপ করা রিকোয়েস্ট পে-লোডের জন্য বেস ক্লাস (এটি সাবক্লাস করুন!)
- `RequestResponse` - মানব প্রতিক্রিয়াকে মূল রিকোয়েস্টের সাথে সংযুক্ত করে

**গুরুত্বপূর্ণ বোঝাপড়া:**
- `RequestInfoExecutor` নিজে ইনপুট সংগ্রহ করে না - এটি শুধুমাত্র ওয়ার্কফ্লো থামায়
- আপনার অ্যাপ্লিকেশন কোডকে `RequestInfoEvent` শুনতে হবে এবং ইনপুট সংগ্রহ করতে হবে
- আপনাকে `send_responses_streaming()` কল করতে হবে একটি ডিকশনারি দিয়ে যা `request_id` কে ব্যবহারকারীর উত্তর এর সাথে ম্যাপ করে

#### 2. **স্ট্রিমিং এক্সিকিউশন প্যাটার্ন**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **ইভেন্ট-চালিত আর্কিটেকচার**
ওয়ার্কফ্লো নিয়ন্ত্রণ করতে নির্দিষ্ট ইভেন্ট শুনুন:
- `RequestInfoEvent` - মানব ইনপুট প্রয়োজন (ওয়ার্কফ্লো থামানো হয়েছে)
- `WorkflowOutputEvent` - চূড়ান্ত ফলাফল উপলব্ধ (ওয়ার্কফ্লো সম্পন্ন)
- `WorkflowStatusEvent` - স্টেট পরিবর্তন (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, ইত্যাদি)

#### 4. **@handler দিয়ে কাস্টম এক্সিকিউটর তৈরি**
`DecisionManager` দেখায় কিভাবে এক্সিকিউটর তৈরি করবেন যা:
- `@handler` ডেকোরেটর ব্যবহার করে মেথডগুলোকে ওয়ার্কফ্লো স্টেপ হিসেবে প্রকাশ করে
- টাইপ করা মেসেজ গ্রহণ করে (যেমন, `RequestResponse[HumanFeedbackRequest, str]`)
- ওয়ার্কফ্লো রাউট করে অন্য এক্সিকিউটরে মেসেজ পাঠিয়ে
- `WorkflowContext` এর মাধ্যমে কন্টেক্সট অ্যাক্সেস করে

#### 5. **মানব সিদ্ধান্তের মাধ্যমে শর্তযুক্ত রাউটিং**
আপনি শর্ত ফাংশন তৈরি করতে পারেন যা মানব প্রতিক্রিয়া মূল্যায়ন করে:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 বাস্তব জীবনের প্রয়োগ:

1. **অনুমোদন ওয়ার্কফ্লো**
   - খরচ রিপোর্ট প্রক্রিয়াকরণের আগে ম্যানেজারের অনুমোদন নিন
   - স্বয়ংক্রিয় ইমেল পাঠানোর আগে মানব পর্যালোচনা প্রয়োজন
   - উচ্চ-মূল্যের লেনদেন সম্পাদনের আগে নিশ্চিত করুন

2. **কন্টেন্ট মডারেশন**
   - সন্দেহজনক কন্টেন্ট মানব পর্যালোচনার জন্য ফ্ল্যাগ করুন
   - এজ কেসে চূড়ান্ত সিদ্ধান্ত নিতে মডারেটরদের জিজ্ঞাসা করুন
   - AI এর আত্মবিশ্বাস কম হলে মানবদের কাছে পাঠান

3. **গ্রাহক সেবা**
   - AI কে রুটিন প্রশ্ন স্বয়ংক্রিয়ভাবে পরিচালনা করতে দিন
   - জটিল সমস্যাগুলো মানব এজেন্টদের কাছে পাঠান
   - গ্রাহককে জিজ্ঞাসা করুন তারা কি মানুষের সাথে কথা বলতে চান

4. **ডেটা প্রসেসিং**
   - অস্পষ্ট ডেটা এন্ট্রি সমাধানের জন্য মানবদের জিজ্ঞাসা করুন
   - অস্পষ্ট ডকুমেন্টের AI ব্যাখ্যা নিশ্চিত করুন
   - একাধিক বৈধ ব্যাখ্যার মধ্যে ব্যবহারকারীদের পছন্দ করতে দিন

5. **নিরাপত্তা-গুরুত্বপূর্ণ সিস্টেম**
   - অপরিবর্তনীয় কাজের আগে মানব নিশ্চিতকরণ প্রয়োজন
   - সংবেদনশীল ডেটা অ্যাক্সেসের আগে অনুমোদন নিন
   - নিয়ন্ত্রিত শিল্পে (স্বাস্থ্যসেবা, অর্থনীতি) সিদ্ধান্ত নিশ্চিত করুন

6. **ইন্টারঅ্যাকটিভ এজেন্ট**
   - কথোপকথন বট তৈরি করুন যা ফলো-আপ প্রশ্ন করে
   - উইজার্ড তৈরি করুন যা ব্যবহারকারীদের জটিল প্রক্রিয়ার মাধ্যমে গাইড করে
   - এজেন্ট ডিজাইন করুন যা মানুষের সাথে ধাপে ধাপে সহযোগিতা করে

### 🔄 তুলনা: Human-in-the-Loop সহ এবং ছাড়া

| বৈশিষ্ট্য | শর্তযুক্ত ওয়ার্কফ্লো | Human-in-the-Loop ওয়ার্কফ্লো |
|-----------|---------------------|---------------------------|
| **এক্সিকিউশন** | একক `workflow.run()` | লুপ `run_stream()` + `send_responses_streaming()` |
| **ব্যবহারকারীর ইনপুট** | নেই (সম্পূর্ণ স্বয়ংক্রিয়) | ইন্টারঅ্যাকটিভ প্রম্পট `input()` বা UI এর মাধ্যমে |
| **উপাদান** | এজেন্ট + এক্সিকিউটর | + RequestInfoExecutor + DecisionManager |
| **ইভেন্ট** | শুধুমাত্র AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, ইত্যাদি |
| **থামানো** | থামানো নেই | ওয়ার্কফ্লো RequestInfoExecutor এ থামে |
| **মানব নিয়ন্ত্রণ** | মানব নিয়ন্ত্রণ নেই | মানবরা গুরুত্বপূর্ণ সিদ্ধান্ত নেয় |
| **ব্যবহার ক্ষেত্র** | অটোমেশন | সহযোগিতা ও তদারকি |

### 🚀 উন্নত প্যাটার্ন:

#### একাধিক মানব সিদ্ধান্ত পয়েন্ট
একই ওয়ার্কফ্লোতে একাধিক `RequestInfoExecutor` নোড থাকতে পারে:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### টাইমআউট হ্যান্ডলিং
মানব প্রতিক্রিয়ার জন্য টাইমআউট প্রয়োগ করুন:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### সমৃদ্ধ UI ইন্টিগ্রেশন
`input()` এর পরিবর্তে, ওয়েব UI, Slack, Teams ইত্যাদির সাথে ইন্টিগ্রেট করুন:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### শর্তযুক্ত Human-in-the-Loop
নির্দিষ্ট পরিস্থিতিতে মানব ইনপুটের জন্য জিজ্ঞাসা করুন:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ সেরা অনুশীলন:

1. **RequestInfoMessage সবসময় সাবক্লাস করুন**
   - টাইপ সেফটি এবং ভ্যালিডেশন প্রদান করে
   - UI রেন্ডারিংয়ের জন্য সমৃদ্ধ কন্টেক্সট সক্ষম করে
   - প্রতিটি রিকোয়েস্ট টাইপের উদ্দেশ্য স্পষ্ট করে

2. **বর্ণনামূলক প্রম্পট ব্যবহার করুন**
   - আপনি যা জিজ্ঞাসা করছেন তার কন্টেক্সট অন্তর্ভুক্ত করুন
   - প্রতিটি পছন্দের পরিণতি ব্যাখ্যা করুন
   - প্রশ্নগুলো সহজ এবং স্পষ্ট রাখুন

3. **অপ্রত্যাশিত ইনপুট পরিচালনা করুন**
   - ব্যবহারকারীর প্রতিক্রিয়া যাচাই করুন
   - অবৈধ ইনপুটের জন্য ডিফল্ট প্রদান করুন
   - স্পষ্ট ত্রুটি বার্তা দিন

4. **রিকোয়েস্ট আইডি ট্র্যাক করুন**
   - রিকোয়েস্ট আইডি এবং প্রতিক্রিয়ার মধ্যে সম্পর্ক ব্যবহার করুন
   - ম্যানুয়ালি স্টেট পরিচালনা করার চেষ্টা করবেন না

5. **নন-ব্লকিং ডিজাইন করুন**
   - ইনপুটের জন্য থ্রেড ব্লক করবেন না
   - পুরো সিস্টেমে অ্যাসিঙ্ক প্যাটার্ন ব্যবহার করুন
   - একাধিক ওয়ার্কফ্লো ইনস্ট্যান্স সমর্থন করুন

### 📚 সম্পর্কিত ধারণা:

- **Agent Middleware** - এজেন্ট কল ইন্টারসেপ্ট করুন (পূর্ববর্তী নোটবুক)
- **Workflow State Management** - রানগুলোর মধ্যে ওয়ার্কফ্লো স্টেট সংরক্ষণ করুন
- **Multi-Agent Collaboration** - Human-in-the-loop কে এজেন্ট টিমের সাথে সংযুক্ত করুন
- **Event-Driven Architectures** - ইভেন্ট দিয়ে রিয়াক্টিভ সিস্টেম তৈরি করুন

---

### 🎓 অভিনন্দন!

Microsoft Agent Framework দিয়ে Human-in-the-loop ওয়ার্কফ্লো আপনি দক্ষভাবে শিখেছেন! এখন আপনি জানেন কিভাবে:
- ✅ ওয়ার্কফ্লো থামিয়ে মানব ইনপুট সংগ্রহ করবেন
- ✅ RequestInfoExecutor এবং RequestInfoMessage ব্যবহার করবেন
- ✅ ইভেন্ট দিয়ে স্ট্রিমিং এক্সিকিউশন পরিচালনা করবেন
- ✅ @handler দিয়ে কাস্টম এক্সিকিউটর তৈরি করবেন
- ✅ মানব সিদ্ধান্তের ভিত্তিতে ওয়ার্কফ্লো রাউট করবেন
- ✅ ইন্টারঅ্যাকটিভ AI এজেন্ট তৈরি করবেন যা মানুষের সাথে সহযোগিতা করে

**বিশ্বাসযোগ্য, নিয়ন্ত্রণযোগ্য AI সিস্টেম তৈরি করার জন্য এটি একটি গুরুত্বপূর্ণ প্যাটার্ন!** 🚀



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়ী থাকব না।
